In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import pandas as pd

# Define the URL for the Manipulation Train set
repo_base = "https://raw.githubusercontent.com/ArkadiusDS/MIPD/master/data"
url = f"{repo_base}/manipulation/train.csv"

print(f"Downloading data from: {url}")

try:
    # Load the dataset into a global variable
    df_mipd_manipulation = pd.read_csv(url)
    print(f"Successfully loaded {len(df_mipd_manipulation)} records into 'df_mipd_manipulation'.")

except Exception as e:
    print(f"Error loading dataset: {e}")

df_mipd_manipulation.info()

Successfully loaded 10749 records into 'df_mipd_manipulation'.
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10749 entries, 0 to 10748
Data columns (total 12 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   article              10749 non-null  object
 1   REFERENCE_ERROR      10749 non-null  bool  
 2   WHATABOUTISM         10749 non-null  bool  
 3   STRAWMAN             10749 non-null  bool  
 4   EMOTIONAL_CONTENT    10749 non-null  bool  
 5   CHERRY_PICKING       10749 non-null  bool  
 6   FALSE_CAUSE          10749 non-null  bool  
 7   MISLEADING_CLICKBAI  10749 non-null  bool  
 8   ANECDOTE             10749 non-null  bool  
 9   LEADING_QUESTIONS    10749 non-null  bool  
 10  EXAGGERATION         10749 non-null  bool  
 11  QUOTE_MINING         10749 non-null  bool  
dtypes: bool(11), object(1)
memory usage: 199.6+ KB


In [4]:
system_instruction='''
Jesteś ekspertem ds. weryfikacji informacji i analizy mediów. Twoim zadaniem jest przeanalizowanie podanego artykułu i zidentyfikowanie w nim technik manipulacji.

Dozwolone kategorie manipulacji to:
- REFERENCE_ERROR
- WHATABOUTISM
- STRAWMAN
- EMOTIONAL_CONTENT
- CHERRY_PICKING
- FALSE_CAUSE
- MISLEADING_CLICKBAIT
- ANECDOTE
- LEADING_QUESTIONS
- EXAGGERATION
- QUOTE_MINING

Instrukcja:
1. Przeanalizuj dokładnie treść podanego artykułu.
2. Dopasuj fragmenty tekstu do powyższych definicji.
3. Zwróć wynik WYŁĄCZNIE w formacie JSON z kluczem "discovered_techniques".
4. Jeśli nie wykryto żadnych technik, zwróć pustą listę.
5. Nie generuj żadnych dodatkowych wyjaśnień, wstępów ani komentarzy.

Przykład wyjścia:
{"discovered_techniques": ["EMOTIONAL_CONTENT", "EXAGGERATION"]}
'''

In [5]:
import json

techniques_columns = df_mipd_manipulation.columns[df_mipd_manipulation.columns.get_loc('REFERENCE_ERROR'):df_mipd_manipulation.columns.get_loc('QUOTE_MINING')+1].tolist()
print(f"Defined manipulation techniques columns: {techniques_columns}")

def format_record(row):
    """Przetwarza jeden wiersz DataFrame na format treningowy."""

    # 1. Wyciągnij aktywne techniki (gdzie wartość w kolumnie jest True/1)
    active_techniques = [col for col in techniques_columns if row[col]]

    # 2. Stwórz Target (Output) w formacie JSON z Markdown
    # ensure_ascii=False jest KLUCZOWE dla polskich znaków
    json_content = json.dumps({"discovered_techniques": active_techniques}, ensure_ascii=False)
    output_str = f"```json\n{json_content}\n```"

    # 3. Zwróć słownik w formacie dla Unsloth
    return {
        "instruction": system_instruction.strip(),
        "input": row['article'],  # Zakładam, że kolumna z tekstem to 'article'
        "output": output_str
    }

# Aplikujemy funkcję do DataFrame (tworzy nowy DataFrame z wynikami)
# axis=1 przetwarza wiersz po wierszu
formatted_df = df_mipd_manipulation.apply(format_record, axis=1, result_type='expand')



Defined manipulation techniques columns: ['REFERENCE_ERROR', 'WHATABOUTISM', 'STRAWMAN', 'EMOTIONAL_CONTENT', 'CHERRY_PICKING', 'FALSE_CAUSE', 'MISLEADING_CLICKBAI', 'ANECDOTE', 'LEADING_QUESTIONS', 'EXAGGERATION', 'QUOTE_MINING']


In [ ]:
# Zapis do pliku JSONL
# orient='records' + lines=True tworzy format JSONL
# force_ascii=False zachowuje polskie ogonki w pliku
output_file = "/content/drive/MyDrive/mipd_train.jsonl"
formatted_df.to_json(output_file, orient='records', lines=True, force_ascii=False)

print(f"Zapisano {len(formatted_df)} rekordów do pliku: {output_file}")
print("Przykładowy rekord:", formatted_df.iloc[0].to_dict())

In [ ]:
import pandas as pd

pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', None)

first_record = formatted_df.iloc[2].to_dict()
print(f"Input:\n{first_record['input']}")
print(f"Output:\n{first_record['output']}")

Input:
"Delirium moŋe byæ wczesnym symptomem COVID-19 u starszych osób" "- Majaczenie lub dezorientacja u starszych osób mogą być wczesnym sygnałem COVID-19, nawet jeśli nie widzimy typowych objawów choroby - uważają naukowcy z Uniwersytetu Massachusetts. Wraz z grupą naukowców z kilku innych placówek medycznych w USA przeanalizowali symptomy COVID-19 pojawiające się u starszych hospitalizowanych pacjentów w wieku powyżej 65 lat. Wyniki badaņ opublikowało czasopismo naukowe JAMA Network Open. - Delirium jest powszechne wśród starszych pacjentów oddziałów ratunkowych (SOR) i wiąże się z ciężkim przebiegiem choroby i wyższą śmiertelnością. Często też pozostaje nierozpoznane - czytamy w artykule. - Majaczenie, dezorientacja, nieuwaga, splątanie oraz inne zaburzenia poznawcze są bowiem częstym objawem infekcji u starszych osób. Dzieje się tak dlatego, że układ odpornościowy ludzi w podeszłym wieku inaczej reaguje na infekcję, niż u młodych osób. Naukowcy zadali sobie pytanie, jak często za

In [ ]:
url = f"{repo_base}/manipulation/validation.csv"

print(f"Downloading data from: {url}")

try:
    # Load the dataset into a global variable
    df_mipd_manipulation_val = pd.read_csv(url)
    print(f"Successfully loaded {len(df_mipd_manipulation_val)} records into 'df_mipd_manipulation_val'.")

except Exception as e:
    print(f"Error loading dataset: {e}")

df_mipd_manipulation_val.info()

url = f"{repo_base}/manipulation/test.csv"

print(f"Downloading data from: {url}")

try:
    # Load the dataset into a global variable
    df_mipd_manipulation_test = pd.read_csv(url)
    print(f"Successfully loaded {len(df_mipd_manipulation_test)} records into 'df_mipd_manipulation_test'.")

except Exception as e:
    print(f"Error loading dataset: {e}")

df_mipd_manipulation_test.info()

Successfully loaded 3086 records into 'df_mipd_manipulation_val'.
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3086 entries, 0 to 3085
Data columns (total 12 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   article              3086 non-null   object
 1   REFERENCE_ERROR      3086 non-null   bool  
 2   WHATABOUTISM         3086 non-null   bool  
 3   STRAWMAN             3086 non-null   bool  
 4   EMOTIONAL_CONTENT    3086 non-null   bool  
 5   CHERRY_PICKING       3086 non-null   bool  
 6   FALSE_CAUSE          3086 non-null   bool  
 7   MISLEADING_CLICKBAI  3086 non-null   bool  
 8   ANECDOTE             3086 non-null   bool  
 9   LEADING_QUESTIONS    3086 non-null   bool  
 10  EXAGGERATION         3086 non-null   bool  
 11  QUOTE_MINING         3086 non-null   bool  
dtypes: bool(11), object(1)
memory usage: 57.4+ KB
Successfully loaded 1521 records into 'df_mipd_manipulation_test'.
<class 'pandas.core.fra

In [ ]:
# Aplikujemy funkcję do DataFrame (tworzy nowy DataFrame z wynikami)
# axis=1 przetwarza wiersz po wierszu
formatted_df_val = df_mipd_manipulation_val.apply(format_record, axis=1, result_type='expand')

# Zapis do pliku JSONL
# orient='records' + lines=True tworzy format JSONL
# force_ascii=False zachowuje polskie ogonki w pliku
output_file = "/content/drive/MyDrive/mipd_val.jsonl"
formatted_df_val.to_json(output_file, orient='records', lines=True, force_ascii=False)

print(f"Zapisano {len(formatted_df_val)} rekordów do pliku: {output_file}")
print("Przykładowy rekord:", formatted_df_val.iloc[0].to_dict())


# Aplikujemy funkcję do DataFrame (tworzy nowy DataFrame z wynikami)
# axis=1 przetwarza wiersz po wierszu
formatted_df_test = df_mipd_manipulation_test.apply(format_record, axis=1, result_type='expand')

# Zapis do pliku JSONL
# orient='records' + lines=True tworzy format JSONL
# force_ascii=False zachowuje polskie ogonki w pliku
output_file = "/content/drive/MyDrive/mipd_test.jsonl"
formatted_df_test.to_json(output_file, orient='records', lines=True, force_ascii=False)

print(f"Zapisano {len(formatted_df_test)} rekordów do pliku: {output_file}")
print("Przykładowy rekord:", formatted_df_test.iloc[0].to_dict())

Zapisano 3086 rekordów do pliku: mipd_val.jsonl
Przykładowy rekord: {'instruction': 'Jesteś ekspertem ds. weryfikacji informacji i analizy mediów. Twoim zadaniem jest przeanalizowanie podanego artykułu i zidentyfikowanie w nim technik manipulacji.\n\nDozwolone kategorie manipulacji to:\n- REFERENCE_ERROR\n- WHATABOUTISM\n- STRAWMAN\n- EMOTIONAL_CONTENT\n- CHERRY_PICKING\n- FALSE_CAUSE\n- MISLEADING_CLICKBAIT\n- ANECDOTE\n- LEADING_QUESTIONS\n- EXAGGERATION\n- QUOTE_MINING\n\nInstrukcja:\n1. Przeanalizuj dokładnie treść podanego artykułu.\n2. Dopasuj fragmenty tekstu do powyższych definicji.\n3. Zwróć wynik WYŁĄCZNIE w formacie JSON z kluczem "discovered_techniques".\n4. Jeśli nie wykryto żadnych technik, zwróć pustą listę.\n5. Nie generuj żadnych dodatkowych wyjaśnień, wstępów ani komentarzy.\n\nPrzykład wyjścia:\n{"discovered_techniques": ["EMOTIONAL_CONTENT", "EXAGGERATION"]}', 'input': '"Szczepienia przeciw COVID-19: pora dnia moŋe mieæ znaczenie" "Więcej podobnych tematów na stroni

In [1]:
%%capture
import os, re
if "COLAB_" not in "".join(os.environ.keys()):
    !pip install unsloth
else:
    # Do this only in Colab notebooks! Otherwise use pip install unsloth
    import torch; v = re.match(r"[0-9]{1,}\.[0-9]{1,}", str(torch.__version__)).group(0)
    xformers = "xformers==" + ("0.0.33.post1" if v=="2.9" else "0.0.32.post2" if v=="2.8" else "0.0.29.post3")
    !pip install --no-deps bitsandbytes accelerate {xformers} peft trl triton cut_cross_entropy unsloth_zoo
    !pip install sentencepiece protobuf "datasets==4.3.0" "huggingface_hub>=0.34.0" hf_transfer
    !pip install --no-deps unsloth
!pip install transformers==4.56.2
!pip install --no-deps trl==0.22.2

In [6]:
from unsloth import FastLanguageModel
from google.colab import userdata
import torch

max_seq_length = 16384
base_model_dir = "drive/MyDrive/bielik-4.5b-base"

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = base_model_dir,
    max_seq_length = max_seq_length, # Choose any for long context!
    load_in_4bit = True,  # 4 bit quantization to reduce memory
    load_in_8bit = False, # [NEW!] A bit more accurate, uses 2x memory
    full_finetuning = False, # [NEW!] We have full finetuning now!
    use_gradient_checkpointing = "unsloth",
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2026.1.2: Fast Llama patching. Transformers: 4.56.2.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.0+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.5.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.33.post1. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


In [7]:
def get_token_length(text):
    """Calculates token length using the loaded tokenizer."""
    if not isinstance(text, str):
        return 0
    return len(tokenizer.encode(text))

TOKEN_LIMIT = 16128

print(f"Filtering datasets to ensure combined 'instruction' + 'input' token length is less than {TOKEN_LIMIT} tokens, using the loaded tokenizer.")

Filtering datasets to ensure combined 'instruction' + 'input' token length is less than 16128 tokens, using the loaded tokenizer.


In [9]:
import pandas as pd

# Apply filtering to the training DataFrame
formatted_df['combined_token_length'] = formatted_df.apply(lambda row: get_token_length(row['instruction']) + get_token_length(row['input']), axis=1)
filtered_formatted_df = formatted_df[formatted_df['combined_token_length'] < TOKEN_LIMIT].copy()
print(f"Original formatted_df records: {len(formatted_df)}")
print(f"Filtered formatted_df records (token length < {TOKEN_LIMIT}): {len(filtered_formatted_df)}")

# Drop the temporary 'combined_token_length' column
filtered_formatted_df = filtered_formatted_df.drop(columns=['combined_token_length'])

# Save the filtered training dataset to a new JSONL file
output_file_16k = "/content/drive/MyDrive/mipd_train_16k.jsonl"
filtered_formatted_df.to_json(output_file_16k, orient='records', lines=True, force_ascii=False)

print(f"Zapisano {len(filtered_formatted_df)} rekordów do pliku: {output_file_16k}")
print("Przykładowy rekord z przefiltrowanego zbioru:", filtered_formatted_df.iloc[0].to_dict())

Original formatted_df records: 10749
Filtered formatted_df records (token length < 16128): 10733
Zapisano 10733 rekordów do pliku: /content/drive/MyDrive/mipd_train_16k.jsonl
Przykładowy rekord z przefiltrowanego zbioru: {'instruction': 'Jesteś ekspertem ds. weryfikacji informacji i analizy mediów. Twoim zadaniem jest przeanalizowanie podanego artykułu i zidentyfikowanie w nim technik manipulacji.\n\nDozwolone kategorie manipulacji to:\n- REFERENCE_ERROR\n- WHATABOUTISM\n- STRAWMAN\n- EMOTIONAL_CONTENT\n- CHERRY_PICKING\n- FALSE_CAUSE\n- MISLEADING_CLICKBAIT\n- ANECDOTE\n- LEADING_QUESTIONS\n- EXAGGERATION\n- QUOTE_MINING\n\nInstrukcja:\n1. Przeanalizuj dokładnie treść podanego artykułu.\n2. Dopasuj fragmenty tekstu do powyższych definicji.\n3. Zwróć wynik WYŁĄCZNIE w formacie JSON z kluczem "discovered_techniques".\n4. Jeśli nie wykryto żadnych technik, zwróć pustą listę.\n5. Nie generuj żadnych dodatkowych wyjaśnień, wstępów ani komentarzy.\n\nPrzykład wyjścia:\n{"discovered_techniqu